In [1]:
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [2]:
class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose


    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):
            # rating이 존재하는 index를 기준으로 training
            xi, yi = self._R.nonzero()
            for i, j in zip(xi, yi):
                self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        # predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost/len(xi))


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)

In [3]:
R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0],
    ])

In [6]:
%%time
factorizer = MatrixFactorization(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=100, verbose=True)
factorizer.fit()

Iteration: 10 ; cost = 1.2726
Iteration: 20 ; cost = 0.9258
Iteration: 30 ; cost = 0.7116
Iteration: 40 ; cost = 0.5549
Iteration: 50 ; cost = 0.4344
Iteration: 60 ; cost = 0.3431
Iteration: 70 ; cost = 0.2768
Iteration: 80 ; cost = 0.2305
Iteration: 90 ; cost = 0.1982
Iteration: 100 ; cost = 0.1751
CPU times: user 58.9 ms, sys: 20.7 ms, total: 79.6 ms
Wall time: 62.7 ms


In [7]:
factorizer.get_complete_matrix()

array([[ 1.2553978 , -0.34394282,  2.93131014,  0.94166886,  2.93890282],
       [ 2.02287733, -1.50790255,  3.03403063,  1.07221896,  1.05704924],
       [ 1.03580709,  1.92820347,  3.47766969,  5.02343146,  2.07577644],
       [ 0.98190613,  3.09560505,  4.93608285,  3.96868344,  3.96749518],
       [ 2.2164486 ,  1.38608417,  4.55373006,  3.83718999,  2.42107192],
       [ 4.67275827,  0.87184236,  5.17948154,  4.03986781,  3.3811057 ],
       [ 0.31293771,  1.65027164,  3.99155827,  1.06091009,  4.62642134]])

In [5]:
factorizer.get_complete_matrix()

array([[ 0.90692124,  0.89752228,  4.94770863,  0.97616019,  3.0389928 ],
       [ 2.12566676,  0.97700122,  2.93961152,  1.04074403,  0.93978602],
       [ 0.97398814,  1.97250323,  2.3221965 ,  5.00038328,  2.75158234],
       [ 1.02766015,  1.21800949,  3.63509308,  4.03622414,  3.95192949],
       [ 2.1086353 ,  0.99094976,  4.93972731,  3.99861029,  5.45395098],
       [ 4.89613898,  1.08124674,  5.09875167,  3.982776  ,  5.50447987],
       [ 1.72024275,  2.55723842,  2.63021019,  0.99397341, -0.76858027]])